In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline

# Document Similarity


- Cosine similarity
- Hellinger-Bhattacharya distance
- Okapi BM25 ranking

We will start with loading the necessary dependencies and the corpus of documents on which we will be testing our various metrics, as shown in the following code snippet:

### toy corpus

In [31]:
from normalization import normalize_corpus
from utils import build_feature_matrix
import numpy as np

# load the toy corpus index
toy_corpus = [
    'The sky is blue',
    'The sky is blue and beautiful',
    'Look at the bright blue sky!',
    'Python is a great Programming language',
    'Python and Java are popular Programming languages',
    'Among Programming languages, both Python and Java are the most used in Analytics',
    'The fox is quicker than the lazy dog',
    'The dog is smarter than the fox',
    'The dog, fox and cat are good friends'
]

In [32]:
# normalize and extract features from the toy corpus
norm_corpus = normalize_corpus(toy_corpus, lemmatize=True)

tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpus,
                                                        feature_type='tfidf',
                                                        ngram_range=(1, 1),
                                                        min_df=0.0, max_df=1.0)                                       

In [33]:
print(tfidf_features[0].toarray()[0])

[0.         0.         0.         0.70710678 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.70710678 0.         0.        ]


### query docs

In [34]:
# load the docs for which we will be measuring similarities
query_docs = [
    'The fox is definitely smarter than the dog',
    'Java is a static typed programming language unlike Python',
    'I love to relax under the beautiful blue sky!'
]  

In [35]:
# normalize and extract features from the query corpus
norm_query_docs =  normalize_corpus(query_docs, lemmatize=True)

query_docs_tfidf = tfidf_vectorizer.transform(norm_query_docs)

In [36]:
print(query_docs_tfidf[0].toarray()[0])

[0.         0.         0.         0.         0.         0.
 0.50936532 0.50936532 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.69360936 0.        ]


# COSINE SIMILARITY

Document :
- The document vectors will be the Bag of Words model–based vectors with TF-IDF values instead of term frequencies. 

Cosine similarity : 

$$ cs\left(u,\ v\right)= \cos \left(\theta \right) = \frac{u\cdot v}{\left|\left|u\left|\right|\kern0.5em \left|\right|v\right|\right|} = \frac{{\displaystyle {\sum}_{i=1}^n}{u}_i\ {v}_i}{\sqrt{{\displaystyle {\sum}_{i=1}^n}{u}_i^2}\ \sqrt{{\displaystyle {\sum}_{i=1}^n}{v}_i^2}} $$

In [37]:
def compute_cosine_similarity(doc_features, corpus_features, top_n=3):
    # get document vectors
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    
    # compute similarities
    similarity = np.dot(doc_features, corpus_features.T)
    
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3)) for index in top_docs]
    
    return top_docs_with_score

In [41]:
print ('Document Similarity Analysis using Cosine Similarity')
print ('='*60)
for index, doc in enumerate(query_docs):    
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_cosine_similarity(doc_tfidf, tfidf_features, top_n=2)
    print ('Document',index+1 ,':', doc)
    print ('Top', len(top_similar_docs), 'similar docs:')
    print ('-'*40)
    for doc_index, sim_score in top_similar_docs:
        print ('  Doc num: {}'.format(doc_index+1))
        print ('  Similarity Score: {}'.format(sim_score))
        print ('  Doc : {}'.format(toy_corpus[doc_index]))
        print ('-'*40)
    print ('') 

Document Similarity Analysis using Cosine Similarity
Document 1 : The fox is definitely smarter than the dog
Top 2 similar docs:
----------------------------------------
  Doc num: 8
  Similarity Score: 1.0
  Doc : The dog is smarter than the fox
----------------------------------------
  Doc num: 7
  Similarity Score: 0.426
  Doc : The fox is quicker than the lazy dog
----------------------------------------

Document 2 : Java is a static typed programming language unlike Python
Top 2 similar docs:
----------------------------------------
  Doc num: 5
  Similarity Score: 0.837
  Doc : Python and Java are popular Programming languages
----------------------------------------
  Doc num: 6
  Similarity Score: 0.661
  Doc : Among Programming languages, both Python and Java are the most used in Analytics
----------------------------------------

Document 3 : I love to relax under the beautiful blue sky!
Top 2 similar docs:
----------------------------------------
  Doc num: 2
  Similarity 

# HELLINGER-BHATTACHARYA DISTANCE

The *Hellinger-Bhattacharya distance* (*HB-distance*) is also called the *Hellinger distance* or the *Bhattacharya distance*. 

참고 : 
- https://en.wikipedia.org/wiki/Hellinger_distance
- https://en.wikipedia.org/wiki/Bhattacharyya_distance
- https://stats.stackexchange.com/questions/130432/differences-between-bhattacharyya-distance-and-kl-divergence

**Hellinger-Bhattacharya distance** : 
- We will be using the TF-IDF–based vectors as our document distributions. 
- This makes it discrete distributions because we have specific TF-IDF values for specific feature terms, unlike continuous distributions.
- We can define the Hellinger-Bhattacharya distance mathematically as :
  - $\large hbd\left(u,\ v\right) = \frac{1}{\sqrt{2}}\ \left|\right|\sqrt{u} - \sqrt{v}\left|\right|{}_2 $  
    - where *hbd*(`u`, `v`) denotes the Hellinger-Bhattacharya distance between the document vectors `u` and `v`, 
    - and it is equal to the Euclidean or L2 norm of the difference of the square root of the vectors divided by the square root of 2. 



**HB-distance converted formula** : 

Considering the document vectors `u` and `v` to be discrete with `n` number of features,

we can further expand the above formula into  :

- $ \large {hbd}\left(u,\ v\right) = \frac{1}{\sqrt{2}}\ \sqrt{{\displaystyle \sum_{i=1}^n}{\left(\sqrt{u_i} - \sqrt{v_i}\right)}^2} $
  - where :
    - $ u=\left({u}_1,\ {u}_2,\dots,\ {u}_n\right) $
    - $ v=\left({v}_1,\ {v}_2,\dots,\ {v}_n\right) $

In [48]:
def compute_hellinger_bhattacharya_distance(doc_features, corpus_features, top_n=3):
    # get document vectors                                            
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    
    # compute hb distances
    distance = np.hstack(  np.sqrt(0.5 * np.sum(  np.square(np.sqrt(doc_features) -  np.sqrt(corpus_features)),  axis=1) ) )
    
    # get docs with lowest distance scores                            
    top_docs = distance.argsort()[:top_n]
    top_docs_with_score = [(index, round(distance[index], 3)) for index in top_docs]
    
    return top_docs_with_score 

In [49]:
print ('Document Similarity Analysis using Hellinger-Bhattacharya distance')
print ('='*60)
for index, doc in enumerate(query_docs):
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_hellinger_bhattacharya_distance(doc_tfidf,  tfidf_features,  top_n=2)
    
    print ('Document',index+1 ,':', doc)
    print ('Top', len(top_similar_docs), 'similar docs:')
    print ('-'*40)
    for doc_index, sim_score in top_similar_docs:
        print ('  Doc num: {}'.format(doc_index+1))
        print ('  Distance Score: {}'.format(sim_score))
        print ('  Doc : {}'.format(toy_corpus[doc_index]))
        print ('-'*40)
    print ('')

Document Similarity Analysis using Hellinger-Bhattacharya distance
Document 1 : The fox is definitely smarter than the dog
Top 2 similar docs:
----------------------------------------
  Doc num: 8
  Distance Score: 0.0
  Doc : The dog is smarter than the fox
----------------------------------------
  Doc num: 7
  Distance Score: 0.96
  Doc : The fox is quicker than the lazy dog
----------------------------------------

Document 2 : Java is a static typed programming language unlike Python
Top 2 similar docs:
----------------------------------------
  Doc num: 5
  Distance Score: 0.53
  Doc : Python and Java are popular Programming languages
----------------------------------------
  Doc num: 4
  Distance Score: 0.766
  Doc : Python is a great Programming language
----------------------------------------

Document 3 : I love to relax under the beautiful blue sky!
Top 2 similar docs:
----------------------------------------
  Doc num: 2
  Distance Score: 0.0
  Doc : The sky is blue and b

# OKAPI BM25 RANKING

### Document Ranking

There are several techniques that are quite popular in information retrieval and search engines, including PageRank and Okapi BM25.

The acronym *BM* stands for *best matching*. 

This technique is also known as BM25, but for the sake of completeness I refer to it as Okapi BM25, 
because originally although the concepts behind the BM25 function were merely theoretical, 
the City University in London built the Okapi Information Retrieval system in the 1980s–90s, 
which implemented this technique to retrieve documents on actual real-world data.

### Okapi BM25 
The Okapi BM25 can be formally defined as a **document ranking** and retrieval function based on a **Bag of Words–based** model for retrieving relevant documents based on a user input query.

**BM25 score** :

Assuming we have these, we can mathematically define the BM25 score between these two documents as :
$ \large bm25\left(CD,\ QD\right) = {\displaystyle \sum_{i=1}^n}idf\left({q}_i\right)\cdot \frac{f\left({q}_i,\kern0.5em CD\right) \cdot \left({k}_1+1\right)}{f\left({q}_i,\kern0.5em CD\right) + {k}_1 \cdot \Big(1-b + b \cdot \frac{\left|CD\right|}{avgdl}} $
- the function *bm*25(*CD*, *QD*) computes the BM25 rank or score of the document *CD* based on the query document *QD*.
- The function ${idf}(q_i)$  gives us the *inverse document frequency* (IDF) of the term $q_i$ in the corpus that contains *CD* and from which we want to retrieve the relevant documents.

$\large idf(t)=1 + \log \frac{C}{1+df(t)} $
- **${idf}(t)$** :
  - where `idf(t)` represents the `idf` for the term `t` 
  - and `C` represents the count of the total number of documents in our corpus 
  - and `df(t)` represents the frequency of the number of documents in which the term `t`is present.





## BM25 implementation steps :
There are several steps we must go through to successfully implement and compute BM25 scores for documents:
1. Build a function to get inverse document frequency (IDF) values for terms in corpus.
2. Build a function for computing BM25 scores for query document and corpus documents.
3. Get Bag of Words–based features for corpus documents and query documents.
4. Compute average length of corpus documents and IDFs of the terms in the corpus documents using function from point 1.
5. Compute BM25 scores, rank relevant documents, and fetch the *n* most relevant documents for each query document using the function in point 2.

참고 :
- [BM25 vs Lucene Default Similarity](https://www.elastic.co/kr/blog/found-bm-vs-lucene-default-similarity)

#### step1 : idf for terms in corpus

In [58]:
import scipy.sparse as sp

def compute_corpus_term_idfs(corpus_features, norm_corpus):
    dfs = np.diff(sp.csc_matrix(corpus_features, copy=True).indptr)
    dfs = 1 + dfs # to smoothen idf later
    total_docs = 1 + len(norm_corpus)
    idfs = 1.0 + np.log(float(total_docs) / dfs)
    return idfs

#### step2 : bm25 similarity

In [59]:
def compute_bm25_similarity(doc_features, corpus_features, corpus_doc_lengths, avg_doc_length, term_idfs, k1=1.5, b=0.75, top_n=3):
    # get corpus bag of words features
    corpus_features = corpus_features.toarray()
    
    # convert query document features to binary features
    # this is to keep a note of which terms exist per document
    doc_features = doc_features.toarray()[0]
    doc_features[doc_features >= 1] = 1
    
    # compute the document idf scores for present terms
    doc_idfs = doc_features * term_idfs
    
    # compute numerator expression in BM25 equation
    numerator_coeff = corpus_features * (k1 + 1)
    numerator = np.multiply(doc_idfs, numerator_coeff)
    
    # compute denominator expression in BM25 equation
    denominator_coeff =  k1 * (1 - b + (b * (corpus_doc_lengths / avg_doc_length)))
    denominator_coeff = np.vstack(denominator_coeff)
    denominator = corpus_features + denominator_coeff
    
    # compute the BM25 score combining the above equations
    bm25_scores = np.sum(np.divide(numerator, denominator), axis=1)
    
    # get top n relevant docs with highest BM25 score                     
    top_docs = bm25_scores.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(bm25_scores[index], 3)) for index in top_docs]
    
    return top_docs_with_score

In [60]:
# step3
vectorizer, corpus_features = build_feature_matrix(norm_corpus, feature_type='frequency')
query_docs_features = vectorizer.transform(norm_query_docs)

# step4
doc_lengths = [len(doc.split()) for doc in norm_corpus]   
avg_dl = np.average(doc_lengths) 
corpus_term_idfs = compute_corpus_term_idfs(corpus_features, norm_corpus)

In [61]:
# step5                 
print ('Document Similarity Analysis using BM25')
print ('='*60)
for index, doc in enumerate(query_docs):    
    doc_features = query_docs_features[index]
    top_similar_docs = compute_bm25_similarity(doc_features,
                                               corpus_features,
                                               doc_lengths,
                                               avg_dl,
                                               corpus_term_idfs,
                                               k1=1.5, b=0.75,
                                               top_n=2)
    print ('Document',index+1 ,':', doc)
    print ('Top', len(top_similar_docs), 'similar docs:')
    print ('-'*40)
    for doc_index, sim_score in top_similar_docs:
        print ('  Doc num: {}'.format(doc_index+1))
        print ('  BM25 Score: {}'.format(sim_score))
        print ('  Doc : {}'.format(toy_corpus[doc_index]))
        print ('-'*40)
    print ('')

Document Similarity Analysis using BM25
Document 1 : The fox is definitely smarter than the dog
Top 2 similar docs:
----------------------------------------
  Doc num: 8
  BM25 Score: 7.334
  Doc : The dog is smarter than the fox
----------------------------------------
  Doc num: 7
  BM25 Score: 3.88
  Doc : The fox is quicker than the lazy dog
----------------------------------------

Document 2 : Java is a static typed programming language unlike Python
Top 2 similar docs:
----------------------------------------
  Doc num: 5
  BM25 Score: 7.248
  Doc : Python and Java are popular Programming languages
----------------------------------------
  Doc num: 6
  BM25 Score: 6.042
  Doc : Among Programming languages, both Python and Java are the most used in Analytics
----------------------------------------

Document 3 : I love to relax under the beautiful blue sky!
Top 2 similar docs:
----------------------------------------
  Doc num: 2
  BM25 Score: 7.334
  Doc : The sky is blue and b